---

# 0. Imports

---

In [1]:
# Basic imports
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt

# Functional imports
import json
#import pyarrow
from pickle import dump
import gc

# Sklearn basic imports
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# XGBoost imports
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score

pd.set_option('display.max_columns', None)

---

# 1. Data Collection

---

In [2]:
#df_2018 = pd.read_parquet("../data/raw/Combined_Flights_2018.parquet", engine="pyarrow")
#df_2019 = pd.read_parquet("../data/raw/Combined_Flights_2019.parquet", engine="pyarrow")
#df_2020 = pd.read_parquet("../data/raw/Combined_Flights_2020.parquet", engine="pyarrow")
df_2021 = pd.read_parquet("../data/raw/Combined_Flights_2021.parquet", engine="pyarrow")
df_2022 = pd.read_parquet("../data/raw/Combined_Flights_2022.parquet", engine="pyarrow")

In [3]:
#data_frames = [df_2018, df_2019, df_2020, df_2021, df_2022]
data_frames = [df_2021, df_2022]
df = pd.concat(data_frames, ignore_index=True)

In [4]:
del df_2021, df_2022, data_frames

gc.collect()

0

In [5]:
df.head(3)

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestCityName,DestState,DestStateFips,DestStateName,DestWac,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2021-03-03,SkyWest Airlines Inc.,SGU,PHX,False,False,724,714.0,0.0,-10.0,818.0,0.0,49.0,79.0,64.0,262.0,2021,1,3,3,3,AA,AA_CODESHARE,19805,AA,3133,OO,20304,OO,N728SK,3133,14794,1479405,34794,"St. George, UT",UT,49,Utah,87,14107,1410702,30466,"Phoenix, AZ",AZ,4,Arizona,81,0.0,-1.0,0700-0759,10.0,724.0,813.0,5.0,843,-25.0,0.0,-2.0,0800-0859,2,0.0
1,2021-03-03,SkyWest Airlines Inc.,PHX,SGU,False,False,922,917.0,0.0,-5.0,1031.0,0.0,48.0,78.0,74.0,262.0,2021,1,3,3,3,AA,AA_CODESHARE,19805,AA,3134,OO,20304,OO,N752SK,3134,14107,1410702,30466,"Phoenix, AZ",AZ,4,Arizona,81,14794,1479405,34794,"St. George, UT",UT,49,Utah,87,0.0,-1.0,0900-0959,23.0,940.0,1028.0,3.0,1040,-9.0,0.0,-1.0,1000-1059,2,0.0
2,2021-03-03,SkyWest Airlines Inc.,MHT,ORD,False,False,1330,1321.0,0.0,-9.0,1501.0,0.0,129.0,180.0,160.0,843.0,2021,1,3,3,3,AA,AA_CODESHARE,19805,AA,3135,OO,20304,OO,N766SK,3135,13296,1329604,30721,"Manchester, NH",NH,33,New Hampshire,14,13930,1393007,30977,"Chicago, IL",IL,17,Illinois,41,0.0,-1.0,1300-1359,15.0,1336.0,1445.0,16.0,1530,-29.0,0.0,-2.0,1500-1559,4,0.0


---

# 2. Exploration and Data cleaning

---

In [6]:
print('Our dataframe contains {} rows and it has {} features.'.format(len(df), df.shape[1]))

Our dataframe contains 10390189 rows and it has 61 features.


In [7]:
print(df.columns)

Index(['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted',
       'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime',
       'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime',
       'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName',
       'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac',
       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName',
       'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15',
       'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOu

## 2.1 Clasificación de los features

### a. Información redundante o irrelevante para el objetivo
Estas columnas no aportan valor predictivo o contienen información que no estará disponible antes del despegue.

| **Columna(s)**                                                                                                         | **Descripción**                                                                                              |
|------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------|
| **'DepTime', 'DepDelayMinutes', 'DepDelay'**                                                                             | Dependen del retraso real al momento de la salida, que es lo que queremos predecir.                         |
| **'ArrTime', 'ArrDelayMinutes', 'AirTime', 'ActualElapsedTime', 'ArrDelay', 'ArrDel15', 'ArrivalDelayGroups'**           | Información posterior al despegue.                                                                          |
| **'Cancelled', 'Diverted', 'DivAirportLandings'**                                                                        | Indican si el vuelo fue cancelado o desviado, lo cual no es útil para predecir retrasos antes del despegue.  |
| **'WheelsOff', 'WheelsOn', 'TaxiOut', 'TaxiIn'**                                                                         | Información posterior al despegue.                                                                          |
| **'Tail_Number'**                                                                                                        | No aporta información relevante y puede introducir ruido.                                                   |
| **'OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID'**| IDs redundantes que pueden ser reemplazados con información más significativa como el nombre del aeropuerto o la ciudad. |
| **'OriginWac', 'DestWac'**                                                                                               | Códigos geográficos redundantes.                                                                            |



### b. Información que podría ser útil pero requiere validación
Estas columnas pueden tener correlación con retrasos, pero debemos evaluar su utilidad.

| **Columna(s)**                                                                                                         | **Descripción**                                                                                              |
|------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------|
| **'FlightDate'**                                                                                                         | Podría ser relevante si extraemos características como feriados, temporada alta, etc.                       |
| **'Flight_Number_Marketing_Airline', 'Flight_Number_Operating_Airline'**                                                 | Dato granular que podría ser redundante con la aerolínea y menos práctico para usuarios en una página web.   |
| **'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners'**                                               | Podrían correlacionarse con retrasos, pero hay que evitar redundancia con otras columnas relacionadas a la aerolínea. |
| **'DOT_ID_Marketing_Airline', 'DOT_ID_Operating_Airline', 'IATA_Code_Marketing_Airline', 'IATA_Code_Operating_Airline'** | IDs redundantes con el nombre de la aerolínea.                                                              |
| **'DistanceGroup'**                                                                                                      | Es categórica, pero podría ser redundante con **'Distance'**.                                                 |
| **'DepartureDelayGroups'**                                                                                               | Representa rangos de retrasos, pero depende del retraso real, por lo que no será útil.                      |



### c. Información clave para el modelo
Estas columnas probablemente tendrán mayor impacto en la predicción.

| **Columna(s)**                                                                 | **Descripción**                                                                                         |
|--------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------|
| **'Airline', 'Operating_Airline'**                                               | Las aerolíneas pueden tener diferentes tasas de puntualidad.                                           |
| **'Origin', 'Dest', 'OriginCityName', 'DestCityName', 'OriginState', 'DestState'**| Lugares de origen y destino son cruciales, ya que algunos aeropuertos tienen mayores tasas de retraso. |
| **'CRSDepTime', 'CRSArrTime', 'DepTimeBlk', 'ArrTimeBlk'**                       | Hora programada y bloque horario pueden correlacionarse con la congestión del aeropuerto y los retrasos.|
| **'Distance'**                                                                   | La distancia entre el origen y el destino puede afectar la probabilidad de retrasos.                   |
| **'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek'**                        | Información temporal que puede capturar tendencias estacionales o semanales.                           |



## 2.2 Propuesta de columnas finales
Variables seleccionadas para el modelo.

| **Columna(s)**             | **Descripción**                               |
|----------------------------|-----------------------------------------------|
| **'Airline' (o 'Operating_Airline')** | Aerolínea del vuelo.                         |
| **'Origin'**                 | Aeropuerto de origen.                        |
| **'Dest'**                   | Aeropuerto de destino.                       |
| **'OriginCityName'**         | Ciudad de origen.                            |
| **'DestCityName'**           | Ciudad de destino.                           |
| **'OriginState'**            | Estado de origen.                            |
| **'DestState'**              | Estado de destino.                           |
| **'CRSDepTime'**             | Hora programada de salida.                   |
| **'DepTimeBlk'**             | Bloque horario de salida.                    |
| **'Distance'**               | Distancia entre origen y destino.            |
| **'Year'**                   | Año del vuelo.                               |
| **'Quarter'**                | Trimestre del vuelo.                         |
| **'Month'**                  | Mes del vuelo.                               |
| **'DayofMonth'**             | Día del mes del vuelo.                       |
| **'DayOfWeek'**              | Día de la semana del vuelo.                  |


## 2.3 Eliminating Irrelevant Features and dealing with null values

Eliminaremos algunos features antes de continuar con nuestro EDA para ahorrar en recursos y tiempo

In [8]:
columnas_finales = [
    # Features
    'Airline', 'Origin', 'Dest', 'OriginCityName', 'DestCityName',
    'OriginState', 'DestState', 'CRSDepTime', 'DepTimeBlk', 'Distance',
    'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
    
    # Targets
    'DepDel15', 'DepDelayMinutes'
]

In [9]:
df = df[columnas_finales]
df.head(3)

,Airline,Origin,Dest,OriginCityName,DestCityName,OriginState,DestState,CRSDepTime,DepTimeBlk,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,DepDel15,DepDelayMinutes
0,SkyWest Airlines Inc.,SGU,PHX,"St. George, UT","Phoenix, AZ",UT,AZ,724,0700-0759,262.0,2021,1,3,3,3,0.0,0.0
1,SkyWest Airlines Inc.,PHX,SGU,"Phoenix, AZ","St. George, UT",AZ,UT,922,0900-0959,262.0,2021,1,3,3,3,0.0,0.0
2,SkyWest Airlines Inc.,MHT,ORD,"Manchester, NH","Chicago, IL",NH,IL,1330,1300-1359,843.0,2021,1,3,3,3,0.0,0.0


In [10]:
df.shape

(10390189, 17)

In [11]:
df.dtypes

Airline             object
Origin              object
Dest                object
OriginCityName      object
DestCityName        object
OriginState         object
DestState           object
CRSDepTime           int64
DepTimeBlk          object
Distance           float64
Year                 int64
Quarter              int64
Month                int64
DayofMonth           int64
DayOfWeek            int64
DepDel15           float64
DepDelayMinutes    float64
dtype: object

## 2.3 Analisis de nuestra dataframe

In [12]:
df.describe()

,CRSDepTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,DepDel15,DepDelayMinutes
count,1.039019e+07,1.039019e+07,1.039019e+07,1.039019e+07,1.039019e+07,1.039019e+07,1.039019e+07,1.016128e+07,1.016128e+07
mean,1.326164e+03,7.964749e+02,2.021393e+03,2.293315e+00,5.837063e+00,1.576069e+01,4.010493e+00,1.908129e-01,1.402861e+01
std,4.806621e+02,5.865190e+02,4.883106e-01,1.047474e+00,3.189623e+00,8.776266e+00,2.006577e+00,3.929419e-01,4.937651e+01
min,1.000000e+00,3.100000e+01,2.021000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,9.170000e+02,3.690000e+02,2.021000e+03,1.000000e+00,3.000000e+00,8.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00
50%,1.319000e+03,6.460000e+02,2.021000e+03,2.000000e+00,6.000000e+00,1.600000e+01,4.000000e+00,0.000000e+00,0.000000e+00
75%,1.729000e+03,1.034000e+03,2.022000e+03,3.000000e+00,8.000000e+00,2.300000e+01,6.000000e+00,0.000000e+00,7.000000e+00
max,2.359000e+03,5.812000e+03,2.022000e+03,4.000000e+00,1.200000e+01,3.100000e+01,7.000000e+00,1.000000e+00,7.223000e+03


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10390189 entries, 0 to 10390188
Data columns (total 17 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Airline          object 
 1   Origin           object 
 2   Dest             object 
 3   OriginCityName   object 
 4   DestCityName     object 
 5   OriginState      object 
 6   DestState        object 
 7   CRSDepTime       int64  
 8   DepTimeBlk       object 
 9   Distance         float64
 10  Year             int64  
 11  Quarter          int64  
 12  Month            int64  
 13  DayofMonth       int64  
 14  DayOfWeek        int64  
 15  DepDel15         float64
 16  DepDelayMinutes  float64
dtypes: float64(3), int64(6), object(8)
memory usage: 1.3+ GB


***Coclusion:***

Nuestro data frame contiene los siguientes tipos de datos:

- ****Float****     --> 3
- ****Integer****   --> 6
- ****String****    --> 8



## 2.4 Buscando duplicados, valores vacios y lidiando con ellos

In [14]:
#Encontrando valores duplicados
print(f"Valores duplicados: {df.duplicated().sum()}")

#Eliminando valores duplicados
df = df.drop_duplicates()
print("\nValores duplicados eliminados!")

Valores duplicados: 1879

Valores duplicados eliminados!


In [15]:
#Encontrando valores NaN
print(f"Valores Nan encontrados: \nColumnas          Cantidad\n{df.isna().sum()[df.isna().sum()>0]}")

df = df.dropna()
print("\nValores NaN eliminados!")

Valores Nan encontrados: 
Columnas          Cantidad
DepDel15           228885
DepDelayMinutes    228885
dtype: int64

Valores NaN eliminados!


***Conclusion:***

De aproximadamente 4M de filas se encontrarons solo 573 filas duplicadas y 762978 de valores nulos

Sabiendo estos numeros lo que haremos seremos eliminarlos para que no afecte en el analisis de datos. Esta decision fue tomada considerando que al ser un porcentaje bajo de data no afectara al momento del analisis

---

# 3. Dividing our dataset in umercial and categorical, Encoding and dividing our dataframe for the clasiffication and regression model

## 3.1 Categorical and numerical

In [16]:
# Understanding how many type of features we have
print(f"Estos son los tipos de datos que tenemos en nuestro data set: {df.dtypes.unique()}")

Estos son los tipos de datos que tenemos en nuestro data set: [dtype('O') dtype('int64') dtype('float64')]


In [17]:
num = ["float64", "int64"]
cat = ['O']

df_num = df.select_dtypes(num)
df_cat = df.select_dtypes(cat)

## 3.2 Encoding our data set

In [18]:
# Creating encoders for categorical features and saving them as JSON files. All files prefixed with 'enc'
# contain the encoding dictionaries for each categorical feature.
df_enc = df.copy()

for column in df_cat.columns:
    unique_values = list(df_cat[column].unique())
    globals()[f"{column}_enc"] = dict(zip(unique_values, range(len(unique_values))))

    json.dump(globals()[f"{column}_enc"], open(f'../data/interim/enc_{column}.json', 'w'))


In [19]:
# Replacing the values in our categorical features to our encoded values (numerical)
for column in df_cat.columns:
    df_enc[column] = df_enc[column].map(json.load(open(f'../data/interim/enc_{column}.json')))

In [20]:
df_enc.head(3)

,Airline,Origin,Dest,OriginCityName,DestCityName,OriginState,DestState,CRSDepTime,DepTimeBlk,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,DepDel15,DepDelayMinutes
0,0,0,0,0,0,0,0,724,0,262.0,2021,1,3,3,3,0.0,0.0
1,0,1,1,1,1,1,1,922,1,262.0,2021,1,3,3,3,0.0,0.0
2,0,2,2,2,2,2,2,1330,2,843.0,2021,1,3,3,3,0.0,0.0


In [21]:
del df, df_cat, df_num

gc.collect()

96

## 3.3 Escalar los datos de distancia y CRSDepTime, para que no valore los datos debido a su reango de diferencia entre ellos

In [22]:
scaler = StandardScaler()
df_enc[['Distance', 'CRSDepTime']] = scaler.fit_transform(df_enc[['Distance', 'CRSDepTime']])

In [23]:
df_enc.head()

,Airline,Origin,Dest,OriginCityName,DestCityName,OriginState,DestState,CRSDepTime,DepTimeBlk,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,DepDel15,DepDelayMinutes
0,0,0,0,0,0,0,0,-1.252175,0,-0.910906,2021,1,3,3,3,0.0,0.0
1,0,1,1,1,1,1,1,-0.839744,1,-0.910906,2021,1,3,3,3,0.0,0.0
2,0,2,2,2,2,2,2,0.010114,2,0.077233,2021,1,3,3,3,0.0,0.0
3,0,3,3,3,3,3,3,0.666255,3,0.119752,2021,1,3,3,3,0.0,0.0
4,0,1,4,1,4,1,4,1.080769,4,-0.633683,2021,1,3,3,3,0.0,0.0


## 3.4 Dividing our data sets para el modelo de clasificacion y el de regresion

In [24]:
#Dataset para modelo de clasificacion
df_clasificacion = df_enc.copy()

#Dataset para modelo de regresion
df_regresion = df_enc[df_enc['DepDel15'] == 1].copy()


print("\nDistribución de la variable objetivo para clasificación (DepDel15):\n")
print(df_clasificacion['DepDel15'].value_counts(normalize=True))

print("\nEstadísticas de la variable objetivo para regresión (DepDelayMinutes):\n")
print(df_regresion['DepDelayMinutes'].describe())


Distribución de la variable objetivo para clasificación (DepDel15):

DepDel15
0.0    0.809152
1.0    0.190848
Name: proportion, dtype: float64

Estadísticas de la variable objetivo para regresión (DepDelayMinutes):

count    1.938903e+06
mean     6.823183e+01
std      9.543036e+01
min      1.500000e+01
25%      2.400000e+01
50%      4.000000e+01
75%      7.700000e+01
max      7.223000e+03
Name: DepDelayMinutes, dtype: float64


## 3.5 Revisar datos procesados


In [25]:
del df_enc

gc.collect()

10

In [26]:
print("Datos procesados para clasificación:")
df_clasificacion.head()

Datos procesados para clasificación:


,Airline,Origin,Dest,OriginCityName,DestCityName,OriginState,DestState,CRSDepTime,DepTimeBlk,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,DepDel15,DepDelayMinutes
0,0,0,0,0,0,0,0,-1.252175,0,-0.910906,2021,1,3,3,3,0.0,0.0
1,0,1,1,1,1,1,1,-0.839744,1,-0.910906,2021,1,3,3,3,0.0,0.0
2,0,2,2,2,2,2,2,0.010114,2,0.077233,2021,1,3,3,3,0.0,0.0
3,0,3,3,3,3,3,3,0.666255,3,0.119752,2021,1,3,3,3,0.0,0.0
4,0,1,4,1,4,1,4,1.080769,4,-0.633683,2021,1,3,3,3,0.0,0.0


In [27]:
print("\nDatos procesados para regresión:")
df_regresion.head()


Datos procesados para regresión:


,Airline,Origin,Dest,OriginCityName,DestCityName,OriginState,DestState,CRSDepTime,DepTimeBlk,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,DepDel15,DepDelayMinutes
36,0,18,13,18,13,11,8,-1.475055,10,-0.970432,2021,1,3,3,3,1.0,22.0
51,0,3,22,3,22,3,0,1.499449,6,0.094240,2021,1,3,3,3,1.0,148.0
59,0,3,10,3,10,3,5,-1.010549,13,-0.164274,2021,1,3,3,3,1.0,18.0
62,0,1,27,1,27,1,8,1.416130,6,0.359558,2021,1,3,3,3,1.0,35.0
66,0,1,28,1,28,1,1,1.078686,4,-0.494221,2021,1,3,3,3,1.0,109.0


---

# 4. Hacer el split de train and test

In [28]:
# Variables predictoras y objetivo para clasificación
X_clasificacion = df_clasificacion.drop(columns=['DepDel15', 'DepDelayMinutes'])
y_clasificacion = df_clasificacion['DepDel15']

# Variables predictoras y objetivo para regresión
X_regresion = df_regresion.drop(columns=['DepDel15', 'DepDelayMinutes'])
y_regresion = df_regresion['DepDelayMinutes']

# Dividir datos para clasificación (80% entrenamiento, 20% prueba)
X_train_clas, X_test_clas, y_train_clas, y_test_clas = train_test_split(
    X_clasificacion, y_clasificacion, test_size=0.2, random_state=42, stratify=y_clasificacion
)

# Dividir datos para regresión (80% entrenamiento, 20% prueba)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_regresion, y_regresion, test_size=0.2, random_state=42
)

# Revisar tamaños de los conjuntos
print("Clasificación:")
print(f"  Tamaño entrenamiento: {X_train_clas.shape}, Tamaño prueba: {X_test_clas.shape}")
print("\nRegresión:")
print(f"  Tamaño entrenamiento: {X_train_reg.shape}, Tamaño prueba: {X_test_reg.shape}")


Clasificación:
  Tamaño entrenamiento: (8127540, 15), Tamaño prueba: (2031885, 15)

Regresión:
  Tamaño entrenamiento: (1551122, 15), Tamaño prueba: (387781, 15)


# 5. Machine Learning

## 5.1 Modelo de clasificacion

In [29]:
'''# 1. Crear el modelo de XGBoost
xgb_clasificador = XGBClassifier(
    objective='binary:logistic',  # Clasificación binaria
    eval_metric='logloss',       # Métrica para evaluar la pérdida
    use_label_encoder=False,     # Deshabilitar codificación automática
    random_state=42
)

# 2. Entrenar el modelo con los datos de entrenamiento
xgb_clasificador.fit(X_train_clas, y_train_clas)

# 3. Predecir en el conjunto de prueba
y_pred_clas = xgb_clasificador.predict(X_test_clas)

# 4. Evaluar el modelo
print("Reporte de clasificación:")
print(classification_report(y_test_clas, y_pred_clas))

print("\nMatriz de confusión:")
print(confusion_matrix(y_test_clas, y_pred_clas))

print("\nExactitud del modelo:")
print(f"{accuracy_score(y_test_clas, y_pred_clas) * 100:.2f}%")
'''

'# 1. Crear el modelo de XGBoost\nxgb_clasificador = XGBClassifier(\n    objective=\'binary:logistic\',  # Clasificación binaria\n    eval_metric=\'logloss\',       # Métrica para evaluar la pérdida\n    use_label_encoder=False,     # Deshabilitar codificación automática\n    random_state=42\n)\n\n# 2. Entrenar el modelo con los datos de entrenamiento\nxgb_clasificador.fit(X_train_clas, y_train_clas)\n\n# 3. Predecir en el conjunto de prueba\ny_pred_clas = xgb_clasificador.predict(X_test_clas)\n\n# 4. Evaluar el modelo\nprint("Reporte de clasificación:")\nprint(classification_report(y_test_clas, y_pred_clas))\n\nprint("\nMatriz de confusión:")\nprint(confusion_matrix(y_test_clas, y_pred_clas))\n\nprint("\nExactitud del modelo:")\nprint(f"{accuracy_score(y_test_clas, y_pred_clas) * 100:.2f}%")\n'

In [30]:
'''from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score

# 1. Aplicar oversampling con SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_clas_bal, y_train_clas_bal = smote.fit_resample(X_train_clas, y_train_clas)

# 2. Crear el modelo con ajuste de ponderación
xgb_clasificador_bal = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    scale_pos_weight=len(y_train_clas[y_train_clas == 0]) / len(y_train_clas[y_train_clas == 1]),  # Ponderación
    random_state=42
)

# 3. Entrenar el modelo con los datos balanceados
xgb_clasificador_bal.fit(X_train_clas_bal, y_train_clas_bal)

# 4. Predecir en el conjunto de prueba
y_pred_clas_bal = xgb_clasificador_bal.predict(X_test_clas)
y_pred_proba_clas_bal = xgb_clasificador_bal.predict_proba(X_test_clas)[:, 1]

# 5. Evaluar el modelo
print("Reporte de clasificación (con balanceo):")
print(classification_report(y_test_clas, y_pred_clas_bal))

print("\nMatriz de confusión (con balanceo):")
print(confusion_matrix(y_test_clas, y_pred_clas_bal))

print("\nExactitud del modelo (con balanceo):")
print(f"{accuracy_score(y_test_clas, y_pred_clas_bal) * 100:.2f}%")

print("\nROC-AUC del modelo:")
print(f"{roc_auc_score(y_test_clas, y_pred_proba_clas_bal):.4f}")
'''

'from imblearn.over_sampling import SMOTE\nfrom sklearn.metrics import roc_auc_score\n\n# 1. Aplicar oversampling con SMOTE al conjunto de entrenamiento\nsmote = SMOTE(random_state=42)\nX_train_clas_bal, y_train_clas_bal = smote.fit_resample(X_train_clas, y_train_clas)\n\n# 2. Crear el modelo con ajuste de ponderación\nxgb_clasificador_bal = XGBClassifier(\n    objective=\'binary:logistic\',\n    eval_metric=\'logloss\',\n    use_label_encoder=False,\n    scale_pos_weight=len(y_train_clas[y_train_clas == 0]) / len(y_train_clas[y_train_clas == 1]),  # Ponderación\n    random_state=42\n)\n\n# 3. Entrenar el modelo con los datos balanceados\nxgb_clasificador_bal.fit(X_train_clas_bal, y_train_clas_bal)\n\n# 4. Predecir en el conjunto de prueba\ny_pred_clas_bal = xgb_clasificador_bal.predict(X_test_clas)\ny_pred_proba_clas_bal = xgb_clasificador_bal.predict_proba(X_test_clas)[:, 1]\n\n# 5. Evaluar el modelo\nprint("Reporte de clasificación (con balanceo):")\nprint(classification_report(y_te

In [31]:
#1. Aplicar oversampling con SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_clas_bal, y_train_clas_bal = smote.fit_resample(X_train_clas, y_train_clas)

# Parámetros para buscar el mejor ajuste
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'scale_pos_weight': [1, len(y_train_clas[y_train_clas == 0]) / len(y_train_clas[y_train_clas == 1])]
}

# Crear el modelo base
xgb_clasificador_tuned = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)

# Búsqueda de hiperparámetros con validación cruzada
grid_search = GridSearchCV(
    estimator=xgb_clasificador_tuned,
    param_grid=param_grid,
    scoring='roc_auc',  # Usamos ROC-AUC como métrica
    cv=3,  # Validación cruzada de 3 pliegues
    verbose=1,
    n_jobs=-1
)

# Entrenar la búsqueda
grid_search.fit(X_train_clas_bal, y_train_clas_bal)

# Mostrar los mejores parámetros
print("Mejores parámetros:", grid_search.best_params_)

# Usar el mejor modelo para predecir
best_model = grid_search.best_estimator_
y_pred_tuned = best_model.predict(X_test_clas)
y_pred_proba_tuned = best_model.predict_proba(X_test_clas)[:, 1]

# Evaluación
print("\nReporte de clasificación (modelo optimizado):")
print(classification_report(y_test_clas, y_pred_tuned))

print("\nMatriz de confusión (modelo optimizado):")
print(confusion_matrix(y_test_clas, y_pred_tuned))

print("\nROC-AUC del modelo optimizado:")
print(f"{roc_auc_score(y_test_clas, y_pred_proba_tuned):.4f}")


'#1. Aplicar oversampling con SMOTE al conjunto de entrenamiento\nsmote = SMOTE(random_state=42)\nX_train_clas_bal, y_train_clas_bal = smote.fit_resample(X_train_clas, y_train_clas)\n\n# Parámetros para buscar el mejor ajuste\nparam_grid = {\n    \'max_depth\': [3, 5, 7],\n    \'learning_rate\': [0.01, 0.1],\n    \'n_estimators\': [100, 200],\n    \'scale_pos_weight\': [1, len(y_train_clas[y_train_clas == 0]) / len(y_train_clas[y_train_clas == 1])]\n}\n\n# Crear el modelo base\nxgb_clasificador_tuned = XGBClassifier(\n    objective=\'binary:logistic\',\n    eval_metric=\'logloss\',\n    use_label_encoder=False,\n    random_state=42\n)\n\n# Búsqueda de hiperparámetros con validación cruzada\ngrid_search = GridSearchCV(\n    estimator=xgb_clasificador_tuned,\n    param_grid=param_grid,\n    scoring=\'roc_auc\',  # Usamos ROC-AUC como métrica\n    cv=3,  # Validación cruzada de 3 pliegues\n    verbose=1,\n    n_jobs=-1\n)\n\n# Entrenar la búsqueda\ngrid_search.fit(X_train_clas_bal, y_trai

In [32]:
'''from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import numpy as np

# Aplicar SMOTE para balancear las clases
smote = SMOTE(random_state=42)
X_train_clas_bal, y_train_clas_bal = smote.fit_resample(X_train_clas, y_train_clas)

# Definir parámetros para la búsqueda aleatoria
param_dist = {
    'max_depth': [3, 5, 7, 9],  # Mayor rango para mayor flexibilidad
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 150, 200],
    'scale_pos_weight': np.linspace(1, 10, 5)  # Valores balanceados automáticamente
}

# Crear el modelo base
xgb_clasificador_tuned = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)

# Búsqueda de hiperparámetros con RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_clasificador_tuned,
    param_distributions=param_dist,
    n_iter=10,  # Número de combinaciones a probar
    scoring='roc_auc',  # Usamos ROC-AUC como métrica
    cv=3,  # Validación cruzada de 3 pliegues
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# Entrenar la búsqueda
random_search.fit(X_train_clas_bal, y_train_clas_bal)

# Mostrar los mejores parámetros
print("Mejores parámetros:", random_search.best_params_)

# Usar el mejor modelo para predecir
best_model = random_search.best_estimator_
y_pred_tuned = best_model.predict(X_test_clas)
y_pred_proba_tuned = best_model.predict_proba(X_test_clas)[:, 1]

# Evaluación
print("\nReporte de clasificación (modelo optimizado):")
print(classification_report(y_test_clas, y_pred_tuned))

print("\nMatriz de confusión (modelo optimizado):")
print(confusion_matrix(y_test_clas, y_pred_tuned))

print("\nROC-AUC del modelo optimizado:")
print(f"{roc_auc_score(y_test_clas, y_pred_proba_tuned):.4f}")
'''

Fitting 3 folds for each of 10 candidates, totalling 30 fits


c:\Users\manue\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:15:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Mejores parámetros: {'scale_pos_weight': 1.0, 'n_estimators': 150, 'max_depth': 9, 'learning_rate': 0.1}

Reporte de clasificación (modelo optimizado):
              precision    recall  f1-score   support

         0.0       0.86      0.85      0.86   1644104
         1.0       0.40      0.43      0.42    387781

    accuracy                           0.77   2031885
   macro avg       0.63      0.64      0.64   2031885
weighted avg       0.78      0.77      0.77   2031885


Matriz de confusión (modelo optimizado):
[[1397626  246478]
 [ 220430  167351]]

ROC-AUC del modelo optimizado:
0.7147
